Notes - Arpit:
- Content based filtering
- find features that will exist with all magazines. maybe paper quality; whether it is concise or expert
- do nlp to get these features out
- fillup with ficitioious or talk abt the attempt 

In [ ]:
# !pip list

In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 5.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633759 sha256=9ed8910c02a8d962a716dade5445b49f960cc105d7beba1bec502851c3e3d9b0
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
!pip install datapane

# Surprise Pkg

- https://www.youtube.com/watch?v=ZspR5PZemcs
- http://surpriselib.com/
- https://towardsdatascience.com/how-you-can-build-simple-recommender-systems-with-surprise-b0d32a8e4802
- https://colab.research.google.com/github/singhsidhukuldeep/Recommendation-System/blob/master/Building_Recommender_System_with_Surprise.ipynb#scrollTo=_G95bJX90FHz

In [ ]:
# %matplotlib inline
import numpy as np
import pandas as pd
import surprise
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Machine Learning- Group Project/Magazine_Subscriptions.csv', names=['asin','reviewerID','rating','timestamp'])

In [ ]:
df1.head(3)

,asin,reviewerID,rating,timestamp
0,B00005N7P0,AH2IFH762VY5U,5.0,1005177600
1,B00005N7P0,AOSFI0JEYU4XM,5.0,1004486400
2,B00005N7OJ,A3JPFWKS83R49V,3.0,1174694400


In [ ]:
df1.nunique()

asin           2428
reviewerID    72098
rating            5
timestamp      5804
dtype: int64

In [ ]:
df1['rating'].value_counts()

5.0    53790
4.0    12676
1.0    11029
3.0     6971
2.0     5223
Name: rating, dtype: int64

In [ ]:
df1.drop(['timestamp'], axis=1, inplace=True)

In [ ]:
df1.shape

(89689, 3)

In [ ]:
print('-Dataset examples- Every 10000th')
print(df1.iloc[::10000, :])

-Dataset examples- Every 10000th
             asin      reviewerID  rating
0      B00005N7P0   AH2IFH762VY5U     5.0
10000  B00005N7SL  A1NCCWFXM0W77X     5.0
20000  B00005NIOH  A3J75FASYN6R43     5.0
30000  B000069YW9   A480UKEZRJY6H     5.0
40000  B00007IJZT   AXV3OJG9BZ2JW     5.0
50000  B000IJ8W8A  A1QNKQ92MZA98A     2.0
60000  B000W3MB5M  A17WA0WNIGXHUG     4.0
70000  B00F418DA4   A33GHRNBXV3CG     5.0
80000  B00007LN6X   A8EVHSC8M32IV     5.0


In [ ]:
df1.groupby('asin')['rating'].count().reset_index().sort_values('rating', ascending=False)[:10]

In [ ]:
df1.groupby('reviewerID')['rating'].count().reset_index().sort_values('rating', ascending=True)[:10]

In [ ]:
df1.groupby('reviewerID')['rating'].count().reset_index().sort_values('rating', ascending=False)[:10]

In [ ]:
users = df1.groupby('reviewerID')['rating'].count().reset_index().sort_values('rating', ascending=False)
users = users[users['rating'] > 5]
users.shape


(411, 2)

In [ ]:
users.sample(10)

### Cases for Prediction - Before deletion

In [ ]:
df1a = df1[['reviewerID','asin','rating']]
df1a['rating'] = df1a['rating'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df1a.shape, df1a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89689 entries, 0 to 89688
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviewerID  89689 non-null  object
 1   asin        89689 non-null  object
 2   rating      89689 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.1+ MB


((89689, 3), None)

In [ ]:
#df.sample() and picked 3 users
df1a[df1a['reviewerID'] == 'AJW4UJUSJVKJA'], df1a[df1a['reviewerID'] == 'A1LESMGKXUXMX5'],df1a[df1a['reviewerID'] == 'A1T17LMQABMBN5']

In [ ]:
#droping 3 ratings to test prediction
df1a.drop(df1a[(df1a['reviewerID'] == 'AJW4UJUSJVKJA') & (df1a['asin'] == 'B00005N7OJ')].index, inplace=True)  #rating 3
df1a.drop(df1a[(df1a['reviewerID'] == 'A1LESMGKXUXMX5') & (df1a['asin'] == 'B00005N7OV')].index, inplace=True) #rating 5
df1a.drop(df1a[(df1a['reviewerID'] == 'A1T17LMQABMBN5') & (df1a['asin'] == 'B00005NIPX')].index, inplace=True) #rating 4

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#check if the deletion worked
df1a[df1a['reviewerID'] == 'AJW4UJUSJVKJA'], df1a[df1a['reviewerID'] == 'A1LESMGKXUXMX5'],df1a[df1a['reviewerID'] == 'A1T17LMQABMBN5']

In [ ]:
#do deleted product have rating from others
df1a[df1a['asin'] == 'B00008GT11'].head(2), df1a[df1a['asin'] == 'B00005N7OV'].head(2),df1a[df1a['asin'] == 'B00005NIPX'].head(2)

(             asin      reviewerID  rating
 40986  B00008GT11  A2JB8BZVUTRHLU     5.0
 40987  B00008GT11  A3OW1EMBTHJ33O     5.0,
            asin      reviewerID  rating
 729  B00005N7OV   AP4CSSLXI0D34     1.0
 730  B00005N7OV  A308V862WSYJ0Z     4.0,
              asin      reviewerID  rating
 22072  B00005NIPX  A1M2T0J45TTE64     4.0
 22073  B00005NIPX  A1IKD6BDEE18CI     5.0)

## Surprise Pkg

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df1a, reader)

trainset = data.build_full_trainset()

algo = NMF()

algo.fit(trainset)

In [ ]:
algo.predict("AJW4UJUSJVKJA", "B00005N7OJ", r_ui=3, verbose=True)

user: AJW4UJUSJVKJA item: B00005N7OJ r_ui = 3.00   est = 3.02   {'was_impossible': False}


Prediction(uid='AJW4UJUSJVKJA', iid='B00005N7OJ', r_ui=3, est=3.0218779025327045, details={'was_impossible': False})

In [ ]:
algo.predict("A1LESMGKXUXMX5", "B00005N7OV", r_ui=5, verbose=True)

user: A1LESMGKXUXMX5 item: B00005N7OV r_ui = 5.00   est = 4.79   {'was_impossible': False}


Prediction(uid='A1LESMGKXUXMX5', iid='B00005N7OV', r_ui=5, est=4.787261440072882, details={'was_impossible': False})

In [ ]:
algo.predict("A1T17LMQABMBN5", "B00005NIPX", r_ui=4, verbose=True)

user: A1T17LMQABMBN5 item: B00005NIPX r_ui = 4.00   est = 3.98   {'was_impossible': False}


Prediction(uid='A1T17LMQABMBN5', iid='B00005NIPX', r_ui=4, est=3.980584084704066, details={'was_impossible': False})

# Attempt 2

- https://github.com/AmolMavuduru/SurprisePythonExamples/blob/master/Recommender%20Systems%20with%20Surprise.ipynb

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import datapane as dp
# import chart_studio.plotly as py
# import chart_studio
# import os
# username = os.environ['PLOTLY_USERNAME']
# api_key = os.environ['PLOTLY_API_KEY']
# chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
%matplotlib inline

In [ ]:
df1['rating'] = df1['rating'].astype(int)

In [ ]:
df1.head()

,asin,reviewerID,rating
0,B00005N7P0,AH2IFH762VY5U,5
1,B00005N7P0,AOSFI0JEYU4XM,5
2,B00005N7OJ,A3JPFWKS83R49V,3
3,B00005N7OJ,A19FKU6JZQ2ECJ,5
4,B00005N7P0,A25MDGOMZ2GALN,5


In [ ]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df1[['reviewerID', 'asin', 'rating']], reader)

In [ ]:
from surprise import SVD
from surprise.model_selection import cross_validate

svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    1.3561  1.3547  1.3678  1.3595  0.0059  
MAE (testset)     1.1035  1.1059  1.1108  1.1067  0.0030  
Fit time          4.86    2.50    2.16    3.17    1.20    
Test time         0.61    0.29    0.20    0.36    0.18    


{'fit_time': (4.859016180038452, 2.495563268661499, 2.161362409591675),
 'test_mae': array([1.10347117, 1.10593586, 1.11079987]),
 'test_rmse': array([1.35606376, 1.35469209, 1.36778948]),
 'test_time': (0.6086664199829102, 0.2889406681060791, 0.19574213027954102)}

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [ ]:
#Predictions
svd.predict(uid='AH2IFH762VY5U', iid='B00005N7P0', r_ui=5)

Prediction(uid='AH2IFH762VY5U', iid='B00005N7P0', r_ui=5, est=4.167391646229564, details={'was_impossible': False})

### Visualization

In [ ]:
svd.qi.shape

(2428, 100)

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, n_iter=500, verbose=3, random_state=1)
books_embedding = tsne.fit_transform(svd.qi)
projection = pd.DataFrame(columns=['x', 'y'], data=books_embedding)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2428 samples in 0.000s...
[t-SNE] Computed neighbors for 2428 samples in 0.214s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2428
[t-SNE] Computed conditional probabilities for sample 2000 / 2428
[t-SNE] Computed conditional probabilities for sample 2428 / 2428
[t-SNE] Mean sigma: 0.261187
[t-SNE] Computed conditional probabilities in 0.127s
[t-SNE] Iteration 50: error = 87.5637436, gradient norm = 0.2069377 (50 iterations in 1.536s)
[t-SNE] Iteration 100: error = 89.0278320, gradient norm = 0.2124482 (50 iterations in 1.570s)
[t-SNE] Iteration 150: error = 89.7798004, gradient norm = 0.2016910 (50 iterations in 1.722s)
[t-SNE] Iteration 200: error = 90.1870575, gradient norm = 0.1864446 (50 iterations in 1.474s)
[t-SNE] Iteration 250: error = 91.4066849, gradient norm = 0.1841513 (50 iterations in 1.558s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 91.406685
[t-SNE] Iteration 300: erro

NameError: ignored

In [ ]:
import plotly.express as px

fig = px.scatter(
    projection, x='x', y='y'
)
fig.show()

# report = dp.Report(dp.Plot(fig) ) #Create a report
# report.publish(name='scatter_plot', open=True, visibility='PUBLIC') #Publish the report